### 📐K-최근접 이웃 알고리즘(K-Nearest Neighbor, KNN)
- 임의의 데이터가 주어지면 그 주변(이웃)의 데이터를 살펴본 뒤 더 많은 데이터가 포함되어 있는 범주로 분류하는 방식이다.
- 가장 간단한 머신러닝 알고리즘으로서, 직관적이고 나름 성능도 괜찮다.
- K를 어떻게 정하냐에 따라 결과 값이 바뀔 수 있다. K는 임의의 데이터가 주어졌을 때 가까운 이웃들의 개수이며, default값은 5이다.  
- K는 가장 가까운 5개의 이웃 데이터를 기반으로 분류하며, 일반적으로 홀수를 사용한다. 짝수일 경우 동점이 되어 하나의 결과를 도출할 수 없기 때문이다.
<div style="width: 80%; display: flex; margin-left: -20px; margin-bottom: 20px;">
    <div>
        <img src="./images/knn01.png">
    </div>
    <div>
        <img src="./images/knn02.png">
    </div>
</div>
- KNN은 fit을 통해 훈련을 시키면, 학습하지 않고 저장만 해놓기 때문에, Lazy Model이라 부른다.
- 새로운 데이터가 주어지면 그제서야 이웃 데이터를 보고 분류해나간다. 따라서 사전 모델링이 필요 없는 real-time 예측이 이루어진다.
- 미리 선을 그어놓거나 나눌 생각이 없기 때문에, SVM이나 선형 회귀보다 빠르다.
<img src="./images/knn03.jpg" width="350" style="margin:10px; margin-left: 0px">
- 데이터와 데이터 사이의 거리를 구해야 더 가까운 클래스로 분류할 수 있으며,  
이는 유클리드 거리(Euclidean Distance)방식과 맨해튼 거리(Manhattan Distance) 방식이 있다(/data-analysis/a_Numpy 에 있는 노름(Norm) 참조).

##### 코로나 바이러스(COVID) 예측

In [3]:
import pandas as pd

corona_df = pd.read_csv('./datasets/corona.csv', low_memory=False)
corona_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278848 entries, 0 to 278847
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Ind_ID               278848 non-null  int64 
 1   Test_date            278848 non-null  object
 2   Cough_symptoms       278596 non-null  object
 3   Fever                278596 non-null  object
 4   Sore_throat          278847 non-null  object
 5   Shortness_of_breath  278847 non-null  object
 6   Headache             278847 non-null  object
 7   Corona               278848 non-null  object
 8   Age_60_above         151528 non-null  object
 9   Sex                  259285 non-null  object
 10  Known_contact        278848 non-null  object
dtypes: int64(1), object(10)
memory usage: 23.4+ MB


##### 결측치 검사

In [4]:
corona_df.isna().sum()

Ind_ID                      0
Test_date                   0
Cough_symptoms            252
Fever                     252
Sore_throat                 1
Shortness_of_breath         1
Headache                    1
Corona                      0
Age_60_above           127320
Sex                     19563
Known_contact               0
dtype: int64

In [12]:
corona_df = corona_df[-corona_df['Cough_symptoms'].isna()]
corona_df = corona_df[-corona_df['Fever'].isna()]
corona_df = corona_df[-corona_df['Sore_throat'].isna()]
corona_df = corona_df[-corona_df['Headache'].isna()]
corona_df['Age_60_above'] = corona_df['Age_60_above'].fillna('No')
corona_df['Sex'] = corona_df['Sex'].fillna('unknown')
corona_df.isna().sum()

Ind_ID                 0
Test_date              0
Cough_symptoms         0
Fever                  0
Sore_throat            0
Shortness_of_breath    0
Headache               0
Corona                 0
Age_60_above           0
Sex                    0
Known_contact          0
dtype: int64

##### 중복행 검사

In [14]:
corona_df.duplicated().sum()

0

In [16]:
corona_df

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex,Known_contact
0,1,11-03-2020,True,False,True,False,False,negative,No,unknown,Abroad
1,2,11-03-2020,False,True,False,False,False,positive,No,unknown,Abroad
2,3,11-03-2020,False,True,False,False,False,positive,No,unknown,Abroad
3,4,11-03-2020,True,False,False,False,False,negative,No,unknown,Abroad
4,5,11-03-2020,True,False,False,False,False,negative,No,unknown,Contact with confirmed
...,...,...,...,...,...,...,...,...,...,...,...
278843,278844,30-04-2020,False,False,False,False,False,positive,No,male,Other
278844,278845,30-04-2020,False,False,False,False,False,negative,No,female,Other
278845,278846,30-04-2020,False,False,False,False,False,negative,No,male,Other
278846,278847,30-04-2020,False,False,False,False,False,negative,No,male,Other


In [23]:
cols = list(corona_df.columns)
cols.remove('Corona')
cols.append('Corona')
corona_df = corona_df[cols]
corona_df = corona_df.rename(columns={"Corona": "Target"})

corona_df

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Age_60_above,Sex,Known_contact,Target
0,1,11-03-2020,True,False,True,False,False,No,unknown,Abroad,negative
1,2,11-03-2020,False,True,False,False,False,No,unknown,Abroad,positive
2,3,11-03-2020,False,True,False,False,False,No,unknown,Abroad,positive
3,4,11-03-2020,True,False,False,False,False,No,unknown,Abroad,negative
4,5,11-03-2020,True,False,False,False,False,No,unknown,Contact with confirmed,negative
...,...,...,...,...,...,...,...,...,...,...,...
278843,278844,30-04-2020,False,False,False,False,False,No,male,Other,positive
278844,278845,30-04-2020,False,False,False,False,False,No,female,Other,negative
278845,278846,30-04-2020,False,False,False,False,False,No,male,Other,negative
278846,278847,30-04-2020,False,False,False,False,False,No,male,Other,negative


In [25]:
corona_df = corona_df[corona_df['Target'] != 'other']

corona_df['Target'].value_counts()
corona_df

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Age_60_above,Sex,Known_contact,Target
0,1,11-03-2020,True,False,True,False,False,No,unknown,Abroad,negative
1,2,11-03-2020,False,True,False,False,False,No,unknown,Abroad,positive
2,3,11-03-2020,False,True,False,False,False,No,unknown,Abroad,positive
3,4,11-03-2020,True,False,False,False,False,No,unknown,Abroad,negative
4,5,11-03-2020,True,False,False,False,False,No,unknown,Contact with confirmed,negative
...,...,...,...,...,...,...,...,...,...,...,...
278843,278844,30-04-2020,False,False,False,False,False,No,male,Other,positive
278844,278845,30-04-2020,False,False,False,False,False,No,female,Other,negative
278845,278846,30-04-2020,False,False,False,False,False,No,male,Other,negative
278846,278847,30-04-2020,False,False,False,False,False,No,male,Other,negative
